<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [ ]:
#import package
!pip install censusgeocode
import csv
import pandas as pd
import censusgeocode as cg

# Data cleaning

In [58]:
data= pd.read_csv("final.csv")
# select the data columns we want 
data = data[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (23,24,25,27,29,30,31,76,92,132,137,139,171,173,197,202,226,227,228,229,253,255,299,300,355,359,360,361,362,379,382,383) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# choose columns status

data.loc[data['PropertySubType'] == 'Single Family Residence' ]
data.loc[data['PropertyType'] == 'Residential']

# Set Input

In [5]:
# input data address
print('Enter address')
address = input()
print('Enter city')
city = input()
print('Enter state')
state = input()
print('Enter zipcode')
zip = input()


Enter address
3700 mapleshade ln
Enter city
plano
Enter state
tx
Enter zipcode
75075


In [52]:
# # limit zipcode range
data = data.loc[data['PostalCode'] == zip ]

In [59]:
# if there is no longitude 
#get Longitude and Latitude

##cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Current_Current')
# example: 1620 Yarborough Drive, city = Sherman, State = TX , zipcode = 75092

result = cg.address(address, city = city, state= state,zipcode = zip)
lo = result[0]['coordinates']['x']
la = result[0]['coordinates']['y']
print(result[0]['coordinates'])



{'x': -96.76240415323872, 'y': 33.00313878557192}


In [76]:
# limit longitude and latitude
data_lo = data[(data['Longitude'] > lo - float(0.01)) & data['Longitude'] < lo + float(0.01)]

data_la = data_lo[(data_lo['Latitude'] > la - float(0.02)) & data_lo['Latitude'] < la + float(0.02)] 


In [78]:
data_lo


,BathroomsFull,BathroomsHalf,BathroomsTotalDecimal,BathroomsTotalInteger,BedroomsTotal,BuildingAreaTotal,City,CloseDate,ClosePrice,CumulativeDaysOnMarket,...,StatusChangeTimestamp,StreetDirPrefix,StreetDirSuffix,StreetName,StreetNumber,StreetNumberNumeric,StreetSuffix,SubdivisionName,USProperty_MUI,YearBuilt


In [66]:
# limit the days




TypeError: ignored

In [ ]:
# limit square feet

In [ ]:
# check number of house = (5,6)个 ok， 不等于repet 上面步骤扩大范围

In [ ]:
# get top 3 mean max min
# return csv